In [ ]:
!pip install mljar-supervised

     |████████████████████████████████| 81kB 7.9MB/s  eta 0:00:01
     |████████████████████████████████| 10.5MB 22.4MB/s 
     |████████████████████████████████| 6.8MB 53.4MB/s 
     |████████████████████████████████| 148.9MB 56kB/s 
     |████████████████████████████████| 1.8MB 44.8MB/s 
     |████████████████████████████████| 66.1MB 48kB/s 
     |████████████████████████████████| 296kB 55.4MB/s 
     |████████████████████████████████| 28.6MB 120kB/s 
     |████████████████████████████████| 63.8MB 53kB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 327kB 55.5MB/s 
     |████████████████████████████████| 225kB 55.1MB/s 
     |████████████████████████████████| 368kB 60.3MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 61kB 10.7MB/s 
  Created wheel for mljar-supervised: filename=mljar_supervised-0.7.20-cp36-none-any.whl size=112327 sha256=b81851e6c4c8f1818be6925d5c349a5c9e0f985ebcef2abf

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML

In [ ]:
train = pd.read_csv('new_train.csv')
train.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,114,fname,label,len,mean_1_1,std_1_1,min_1_1,max_1_1,abs_mean_1_1,abs_std_1_1,abs_min_1_1,abs_max_1_1,diff_mean_1_1,diff_std_1_1,diff_min_1_1,diff_max_1_1,mean_1_2,std_1_2,min_1_2,max_1_2,abs_mean_1_2,abs_std_1_2,abs_min_1_2,abs_max_1_2,diff_mean_1_2,diff_std_1_2,diff_min_1_2,diff_max_1_2,mean_1_3,std_1_3,min_1_3,max_1_3,abs_mean_1_3,abs_std_1_3,abs_min_1_3,abs_max_1_3,diff_mean_1_3,diff_std_1_3,diff_min_1_3,diff_max_1_3
0,-591.031682,107.357705,18.891522,25.387899,15.477197,-0.666428,-11.393417,2.117549,2.066153,1.084113,-1.802101,-0.502671,-1.535924,-2.903620,-1.109281,-3.872477,-3.135773,-2.884538,-3.447222,-4.148853,114.747323,81.358803,22.898075,21.452958,25.076083,16.813999,17.834653,13.000395,8.901351,8.299461,8.969343,9.471980,8.134469,6.744038,6.492289,8.711509,7.025930,5.137653,6.364879,6.571455,...,0.0,346.wav,Negative,99072.0,0.104914,700.909507,-3863.0,3822.0,348.638707,608.050327,0.0,3863.0,0.0,0.0,0.0,0.0,0.750424,523.049834,-2967.0,2294.0,290.560744,434.920850,0.0,2967.0,0.0,0.0,0.0,0.0,0.532219,483.010517,-2967.0,2294.0,237.519985,420.575438,0.0,2967.0,0.0,0.0,0.0,0.0
1,-360.014782,154.619607,-7.848003,4.830051,34.022187,11.741361,-12.634243,-14.327259,0.076565,7.371945,-7.121121,-16.676015,0.160181,5.824123,-8.658678,-11.311229,-1.396284,3.411900,-5.416746,-6.371157,116.788382,79.689070,34.310244,21.750968,22.658344,16.706306,12.549705,12.997490,11.681934,8.341038,10.646312,13.060791,9.547757,8.554503,8.564101,9.607275,7.191809,8.220305,7.871040,9.133367,...,0.0,163.wav,Neutral,114048.0,-0.300286,3872.495473,-31064.0,28249.0,2116.474353,3242.955070,0.0,31064.0,0.0,0.0,0.0,0.0,-1.643764,3074.024163,-16980.0,10070.0,1977.514766,2353.521321,0.0,16980.0,0.0,0.0,0.0,0.0,-0.505051,2474.739419,-15796.0,9437.0,1553.343277,1926.515017,0.0,15796.0,0.0,0.0,0.0,0.0
2,-594.040743,103.790633,13.922286,30.873570,9.980270,-4.311017,-6.491590,5.946820,0.383232,-0.399051,-3.736077,2.666826,2.417960,-4.896455,-4.457233,-7.752620,-4.260378,-2.706262,-5.689548,-5.966477,131.838122,87.268409,32.530141,21.356198,18.830361,18.815664,11.319590,9.396798,6.753247,14.504216,7.928673,7.569636,6.002007,9.334162,10.912878,10.096592,6.524360,5.821844,6.728617,8.242629,...,0.0,288.wav,Negative,74880.0,0.261632,736.231596,-4227.0,3330.0,375.935457,633.016242,0.0,4227.0,0.0,0.0,0.0,0.0,-1.379033,721.293474,-3259.0,3330.0,393.482505,604.514429,0.0,3330.0,0.0,0.0,0.0,0.0,0.979447,323.873973,-2391.0,1519.0,134.837460,294.472696,0.0,2391.0,0.0,0.0,0.0,0.0
3,-565.080365,125.795892,6.180520,25.099404,15.052440,1.483659,-7.802284,0.972381,-2.742347,1.164681,-2.925657,3.055605,1.839463,-6.639771,-6.208944,-7.664477,-4.032501,-2.960873,-4.360971,-5.504611,123.413103,81.588843,32.923758,21.679085,17.567169,14.698030,13.847179,7.279012,7.115872,9.067360,8.662264,7.801159,8.118018,7.631249,7.495879,8.143258,7.420902,5.241293,6.612754,7.062080,...,0.0,279.wav,Negative,104832.0,0.309505,620.150143,-3417.0,3573.0,329.448985,525.404284,0.0,3573.0,0.0,0.0,0.0,0.0,-1.225847,653.198277,-2889.0,2922.0,363.120078,542.967126,0.0,2922.0,0.0,0.0,0.0,0.0,-0.004865,551.390620,-2789.0,2592.0,302.603308,460.936931,0.0,2789.0,0.0,0.0,0.0,0.0
4,-572.168372,127.598331,23.113026,31.622201,4.401347,-7.654443,-8.338925,8.768979,2.916846,1.646538,-4.135653,1.688965,2.759710,-4.706310,-6.717592,-8.303737,-4.042487,-4.562963,-6.388863,-8.681849,122.561119,84.240538,25.244293,23.930681,18.478424,20.105318,14.046924,10.170725,10.608277,7.091116,9.051654,8.001456,7.521787,8.094509,8.837414,8.540785,7.761704,6.376638,7.824119,7.157348,...,0.0,244.wav,Negative,102528.0,0.285278,756.789765,-3357.0,3404.0,428.831646,623.565754,0.0,3404.0,0.0,0.0,0.0,0.0,0.341916,740.113889,-3357.0,3404.0,436.305321,597.834720,0.0,3404.0,0.0,0.0,0.0,0.0,2.616895,686.565335,-3357.0,3023.0,404.438290,554.804900,0.0,3357.0,0.0,0.0,0.0,0.0


In [ ]:
test = pd.read_csv('new_test.csv')


In [ ]:
# map = {'Neutral':0,'Negative':1,'Positive':2}

# train['label'] = train['label'].apply(lambda x:map[x])
# train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,114,fname,label,len,mean_1_1,std_1_1,min_1_1,max_1_1,abs_mean_1_1,abs_std_1_1,abs_min_1_1,abs_max_1_1,diff_mean_1_1,diff_std_1_1,diff_min_1_1,diff_max_1_1,mean_1_2,std_1_2,min_1_2,max_1_2,abs_mean_1_2,abs_std_1_2,abs_min_1_2,abs_max_1_2,diff_mean_1_2,diff_std_1_2,diff_min_1_2,diff_max_1_2,mean_1_3,std_1_3,min_1_3,max_1_3,abs_mean_1_3,abs_std_1_3,abs_min_1_3,abs_max_1_3,diff_mean_1_3,diff_std_1_3,diff_min_1_3,diff_max_1_3
0,-591.031682,107.357705,18.891522,25.387899,15.477197,-0.666428,-11.393417,2.117549,2.066153,1.084113,-1.802101,-0.502671,-1.535924,-2.903620,-1.109281,-3.872477,-3.135773,-2.884538,-3.447222,-4.148853,114.747323,81.358803,22.898075,21.452958,25.076083,16.813999,17.834653,13.000395,8.901351,8.299461,8.969343,9.471980,8.134469,6.744038,6.492289,8.711509,7.025930,5.137653,6.364879,6.571455,...,0.0,346.wav,1,99072.0,0.104914,700.909507,-3863.0,3822.0,348.638707,608.050327,0.0,3863.0,0.0,0.0,0.0,0.0,0.750424,523.049834,-2967.0,2294.0,290.560744,434.920850,0.0,2967.0,0.0,0.0,0.0,0.0,0.532219,483.010517,-2967.0,2294.0,237.519985,420.575438,0.0,2967.0,0.0,0.0,0.0,0.0
1,-360.014782,154.619607,-7.848003,4.830051,34.022187,11.741361,-12.634243,-14.327259,0.076565,7.371945,-7.121121,-16.676015,0.160181,5.824123,-8.658678,-11.311229,-1.396284,3.411900,-5.416746,-6.371157,116.788382,79.689070,34.310244,21.750968,22.658344,16.706306,12.549705,12.997490,11.681934,8.341038,10.646312,13.060791,9.547757,8.554503,8.564101,9.607275,7.191809,8.220305,7.871040,9.133367,...,0.0,163.wav,0,114048.0,-0.300286,3872.495473,-31064.0,28249.0,2116.474353,3242.955070,0.0,31064.0,0.0,0.0,0.0,0.0,-1.643764,3074.024163,-16980.0,10070.0,1977.514766,2353.521321,0.0,16980.0,0.0,0.0,0.0,0.0,-0.505051,2474.739419,-15796.0,9437.0,1553.343277,1926.515017,0.0,15796.0,0.0,0.0,0.0,0.0
2,-594.040743,103.790633,13.922286,30.873570,9.980270,-4.311017,-6.491590,5.946820,0.383232,-0.399051,-3.736077,2.666826,2.417960,-4.896455,-4.457233,-7.752620,-4.260378,-2.706262,-5.689548,-5.966477,131.838122,87.268409,32.530141,21.356198,18.830361,18.815664,11.319590,9.396798,6.753247,14.504216,7.928673,7.569636,6.002007,9.334162,10.912878,10.096592,6.524360,5.821844,6.728617,8.242629,...,0.0,288.wav,1,74880.0,0.261632,736.231596,-4227.0,3330.0,375.935457,633.016242,0.0,4227.0,0.0,0.0,0.0,0.0,-1.379033,721.293474,-3259.0,3330.0,393.482505,604.514429,0.0,3330.0,0.0,0.0,0.0,0.0,0.979447,323.873973,-2391.0,1519.0,134.837460,294.472696,0.0,2391.0,0.0,0.0,0.0,0.0
3,-565.080365,125.795892,6.180520,25.099404,15.052440,1.483659,-7.802284,0.972381,-2.742347,1.164681,-2.925657,3.055605,1.839463,-6.639771,-6.208944,-7.664477,-4.032501,-2.960873,-4.360971,-5.504611,123.413103,81.588843,32.923758,21.679085,17.567169,14.698030,13.847179,7.279012,7.115872,9.067360,8.662264,7.801159,8.118018,7.631249,7.495879,8.143258,7.420902,5.241293,6.612754,7.062080,...,0.0,279.wav,1,104832.0,0.309505,620.150143,-3417.0,3573.0,329.448985,525.404284,0.0,3573.0,0.0,0.0,0.0,0.0,-1.225847,653.198277,-2889.0,2922.0,363.120078,542.967126,0.0,2922.0,0.0,0.0,0.0,0.0,-0.004865,551.390620,-2789.0,2592.0,302.603308,460.936931,0.0,2789.0,0.0,0.0,0.0,0.0
4,-572.168372,127.598331,23.113026,31.622201,4.401347,-7.654443,-8.338925,8.768979,2.916846,1.646538,-4.135653,1.688965,2.759710,-4.706310,-6.717592,-8.303737,-4.042487,-4.562963,-6.388863,-8.681849,122.561119,84.240538,25.244293,23.930681,18.478424,20.105318,14.046924,10.170725,10.608277,7.091116,9.051654,8.001456,7.521787,8.094509,8.837414,8.540785,7.761704,6.376638,7.824119,7.157348,...,0.0,244.wav,1,102528.0,0.285278,756.789765,-3357.0,3404.0,428.831646,623.565754,0.0,3404.0,0.0,0.0,0.0,0.0,0.341916,740.113889,-3357.0,3404.0,436.305321,597.834720,0.0,3404.0,0.0,0.0,0.0,0.0,2.616895,686.565335,-3357.0,3023.0,404.438290,554.804900,0.0,3357.0,0.0,0.0,0.0,0.0


In [ ]:
train['fname'] = train['fname'].apply(lambda x:x.split('.')[0])
train.fname


0      346
1      163
2      288
3      279
4      244
      ... 
245    204
246     46
247    318
248     49
249    191
Name: fname, Length: 250, dtype: object

In [ ]:
train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,114,fname,label,len,mean_1_1,std_1_1,min_1_1,max_1_1,abs_mean_1_1,abs_std_1_1,abs_min_1_1,abs_max_1_1,diff_mean_1_1,diff_std_1_1,diff_min_1_1,diff_max_1_1,mean_1_2,std_1_2,min_1_2,max_1_2,abs_mean_1_2,abs_std_1_2,abs_min_1_2,abs_max_1_2,diff_mean_1_2,diff_std_1_2,diff_min_1_2,diff_max_1_2,mean_1_3,std_1_3,min_1_3,max_1_3,abs_mean_1_3,abs_std_1_3,abs_min_1_3,abs_max_1_3,diff_mean_1_3,diff_std_1_3,diff_min_1_3,diff_max_1_3
0,-591.031682,107.357705,18.891522,25.387899,15.477197,-0.666428,-11.393417,2.117549,2.066153,1.084113,-1.802101,-0.502671,-1.535924,-2.903620,-1.109281,-3.872477,-3.135773,-2.884538,-3.447222,-4.148853,114.747323,81.358803,22.898075,21.452958,25.076083,16.813999,17.834653,13.000395,8.901351,8.299461,8.969343,9.471980,8.134469,6.744038,6.492289,8.711509,7.025930,5.137653,6.364879,6.571455,...,0.0,346,Negative,99072.0,0.104914,700.909507,-3863.0,3822.0,348.638707,608.050327,0.0,3863.0,0.0,0.0,0.0,0.0,0.750424,523.049834,-2967.0,2294.0,290.560744,434.920850,0.0,2967.0,0.0,0.0,0.0,0.0,0.532219,483.010517,-2967.0,2294.0,237.519985,420.575438,0.0,2967.0,0.0,0.0,0.0,0.0
1,-360.014782,154.619607,-7.848003,4.830051,34.022187,11.741361,-12.634243,-14.327259,0.076565,7.371945,-7.121121,-16.676015,0.160181,5.824123,-8.658678,-11.311229,-1.396284,3.411900,-5.416746,-6.371157,116.788382,79.689070,34.310244,21.750968,22.658344,16.706306,12.549705,12.997490,11.681934,8.341038,10.646312,13.060791,9.547757,8.554503,8.564101,9.607275,7.191809,8.220305,7.871040,9.133367,...,0.0,163,Neutral,114048.0,-0.300286,3872.495473,-31064.0,28249.0,2116.474353,3242.955070,0.0,31064.0,0.0,0.0,0.0,0.0,-1.643764,3074.024163,-16980.0,10070.0,1977.514766,2353.521321,0.0,16980.0,0.0,0.0,0.0,0.0,-0.505051,2474.739419,-15796.0,9437.0,1553.343277,1926.515017,0.0,15796.0,0.0,0.0,0.0,0.0
2,-594.040743,103.790633,13.922286,30.873570,9.980270,-4.311017,-6.491590,5.946820,0.383232,-0.399051,-3.736077,2.666826,2.417960,-4.896455,-4.457233,-7.752620,-4.260378,-2.706262,-5.689548,-5.966477,131.838122,87.268409,32.530141,21.356198,18.830361,18.815664,11.319590,9.396798,6.753247,14.504216,7.928673,7.569636,6.002007,9.334162,10.912878,10.096592,6.524360,5.821844,6.728617,8.242629,...,0.0,288,Negative,74880.0,0.261632,736.231596,-4227.0,3330.0,375.935457,633.016242,0.0,4227.0,0.0,0.0,0.0,0.0,-1.379033,721.293474,-3259.0,3330.0,393.482505,604.514429,0.0,3330.0,0.0,0.0,0.0,0.0,0.979447,323.873973,-2391.0,1519.0,134.837460,294.472696,0.0,2391.0,0.0,0.0,0.0,0.0
3,-565.080365,125.795892,6.180520,25.099404,15.052440,1.483659,-7.802284,0.972381,-2.742347,1.164681,-2.925657,3.055605,1.839463,-6.639771,-6.208944,-7.664477,-4.032501,-2.960873,-4.360971,-5.504611,123.413103,81.588843,32.923758,21.679085,17.567169,14.698030,13.847179,7.279012,7.115872,9.067360,8.662264,7.801159,8.118018,7.631249,7.495879,8.143258,7.420902,5.241293,6.612754,7.062080,...,0.0,279,Negative,104832.0,0.309505,620.150143,-3417.0,3573.0,329.448985,525.404284,0.0,3573.0,0.0,0.0,0.0,0.0,-1.225847,653.198277,-2889.0,2922.0,363.120078,542.967126,0.0,2922.0,0.0,0.0,0.0,0.0,-0.004865,551.390620,-2789.0,2592.0,302.603308,460.936931,0.0,2789.0,0.0,0.0,0.0,0.0
4,-572.168372,127.598331,23.113026,31.622201,4.401347,-7.654443,-8.338925,8.768979,2.916846,1.646538,-4.135653,1.688965,2.759710,-4.706310,-6.717592,-8.303737,-4.042487,-4.562963,-6.388863,-8.681849,122.561119,84.240538,25.244293,23.930681,18.478424,20.105318,14.046924,10.170725,10.608277,7.091116,9.051654,8.001456,7.521787,8.094509,8.837414,8.540785,7.761704,6.376638,7.824119,7.157348,...,0.0,244,Negative,102528.0,0.285278,756.789765,-3357.0,3404.0,428.831646,623.565754,0.0,3404.0,0.0,0.0,0.0,0.0,0.341916,740.113889,-3357.0,3404.0,436.305321,597.834720,0.0,3404.0,0.0,0.0,0.0,0.0,2.616895,686.565335,-3357.0,3023.0,404.438290,554.804900,0.0,3357.0,0.0,0.0,0.0,0.0


In [ ]:
test['fname'] = test['fname'].apply(lambda x:x.split('.')[0])
test.fname

0      157
1       26
2       75
3      185
4      136
      ... 
105    182
106    134
107    322
108    165
109     84
Name: fname, Length: 110, dtype: object

In [ ]:
X = train.drop(columns=['label'],axis=1)
y = train['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = train.label,test_size=0.1)


In [ ]:
from sklearn.ensemble import *
from sklearn.model_selection import *
from sklearn.metrics import *

# !pip install catboost
from catboost import CatBoostClassifier

In [ ]:
automl = AutoML(
    algorithms=["Xgboost", "CatBoost","LightGBM"],
    golden_features=True,
    features_selection=False,
    stack_models=True,
    train_ensemble=True,
    explain_level=0,
    mode='Compete',
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 5,
        "shuffle": False,
        "stratify": True,
    }
)
automl.fit(X_train, y_train)
y_pred = automl.predict(X_test)

AutoML directory: AutoML_3
The task is multiclass_classification with evaluation metric logloss
AutoML will use algorithms: ['Xgboost', 'CatBoost', 'LightGBM']
AutoML will stack models
AutoML will ensemble availabe models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'hill_climbing_1', 'hill_climbing_2', 'ensemble', 'stack', 'ensemble_stacked']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 3 models
1_Default_Xgboost logloss 0.152667 trained in 8.53 seconds
2_Default_LightGBM logloss 0.115774 trained in 5.32 seconds
3_Default_CatBoost logloss 0.184185 trained in 163.92 seconds
* Step not_so_random will try to check up to 27 models
4_Xgboost logloss 0.176412 trained in 4.61 seconds
13_LightGBM logloss 0.152113 trained in 2.54 seconds


In [ ]:
from sklearn.metrics import accuracy_score
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}%")

Accuracy: 0.88%


In [ ]:
preds = automl.predict(test.drop(columns=['label'],axis=1))

In [ ]:
x = pd.DataFrame()
x['Filename'] = test.fname
x['Class'] = preds
x.head()

,Filename,Class
0,157,0
1,26,2
2,75,2
3,185,0
4,136,0


In [ ]:
x = x.sort_values(by='Filename')
x.head()

,Filename,Class
99,112.wav,Positive
60,113.wav,Positive
84,115.wav,Positive
44,119.wav,Positive
43,123.wav,Neutral


In [ ]:
x.to_csv('hack_nitp_mljar_v1.csv',index=False)

In [ ]:
model_cat = CatBoostClassifier(od_type='Iter', iterations=10000, task_type='GPU')
model_cat.fit(X_train, y_train.astype(int),
              eval_set=(X_test, y_test.astype(int)),
              early_stopping_rounds=100,
              )

Learning rate set to 0.032869
0:	learn: 1.0679023	test: 1.0754308	best: 1.0754308 (0)	total: 15.6ms	remaining: 2m 35s
1:	learn: 1.0387873	test: 1.0550226	best: 1.0550226 (1)	total: 28.7ms	remaining: 2m 23s
2:	learn: 1.0106223	test: 1.0317801	best: 1.0317801 (2)	total: 48.1ms	remaining: 2m 40s
3:	learn: 0.9839773	test: 1.0091240	best: 1.0091240 (3)	total: 62ms	remaining: 2m 34s
4:	learn: 0.9547039	test: 0.9860462	best: 0.9860462 (4)	total: 75ms	remaining: 2m 29s
5:	learn: 0.9309790	test: 0.9668463	best: 0.9668463 (5)	total: 88.5ms	remaining: 2m 27s
6:	learn: 0.9071142	test: 0.9536228	best: 0.9536228 (6)	total: 106ms	remaining: 2m 30s
7:	learn: 0.8848873	test: 0.9340529	best: 0.9340529 (7)	total: 119ms	remaining: 2m 28s
8:	learn: 0.8631577	test: 0.9175695	best: 0.9175695 (8)	total: 133ms	remaining: 2m 27s
9:	learn: 0.8379414	test: 0.8998118	best: 0.8998118 (9)	total: 146ms	remaining: 2m 25s
10:	learn: 0.8196153	test: 0.8824483	best: 0.8824483 (10)	total: 160ms	remaining: 2m 25s
11:	learn

In [ ]:
importance = pd.DataFrame(data=model_cat.feature_importances_, index=X_train.columns, columns=['imp']).sort_values(by='imp',ascending=False)
imp_feat = importance[importance['imp'] > 0.05].index
imp_feat

Index(['102', 'abs_std_1_1', 'max_1_2', 'max_1_3', 'max_1_1', '92', 'len',
       'std_1_1', 'min_1_2', '9',
       ...
       '37', '24', '25', '39', '5', '86', '89', '14', '4', 'diff_mean_1_2'],
      dtype='object', length=113)

In [ ]:
model_cat = CatBoostClassifier(od_type='Iter', iterations=10000, task_type='GPU')
model_cat.fit(X_train[imp_feat], y_train.astype(int),
              eval_set=(X_test[imp_feat], y_test.astype(int)),
              early_stopping_rounds=200,
              )

Learning rate set to 0.032869
0:	learn: 1.0634024	test: 1.0667436	best: 1.0667436 (0)	total: 14.9ms	remaining: 2m 28s
1:	learn: 1.0291811	test: 1.0410249	best: 1.0410249 (1)	total: 27.4ms	remaining: 2m 17s
2:	learn: 1.0024922	test: 1.0197959	best: 1.0197959 (2)	total: 41.7ms	remaining: 2m 18s
3:	learn: 0.9753356	test: 0.9940085	best: 0.9940085 (3)	total: 54.4ms	remaining: 2m 16s
4:	learn: 0.9520336	test: 0.9744609	best: 0.9744609 (4)	total: 69.5ms	remaining: 2m 18s
5:	learn: 0.9265492	test: 0.9560550	best: 0.9560550 (5)	total: 83.1ms	remaining: 2m 18s
6:	learn: 0.9052298	test: 0.9395031	best: 0.9395031 (6)	total: 95.8ms	remaining: 2m 16s
7:	learn: 0.8867149	test: 0.9280131	best: 0.9280131 (7)	total: 109ms	remaining: 2m 15s
8:	learn: 0.8674003	test: 0.9130749	best: 0.9130749 (8)	total: 125ms	remaining: 2m 19s
9:	learn: 0.8438487	test: 0.8936595	best: 0.8936595 (9)	total: 138ms	remaining: 2m 17s
10:	learn: 0.8235655	test: 0.8775487	best: 0.8775487 (10)	total: 151ms	remaining: 2m 16s
11:	

In [ ]:
prediction = model_cat.predict(test[imp_feat])


In [ ]:
preds = [i[0] for i in prediction]
# preds

In [ ]:
reverse_map = {0:'Neutral',1:'Negative',2:'Positive'}

In [ ]:
sub = pd.DataFrame()
sub['Filename']= test.fname
sub['Class'] = preds
sub['Class'] = sub['Class'].apply(lambda x:reverse_map[x])
sub.head()

,Filename,Class
0,157.wav,Neutral
1,26.wav,Positive
2,75.wav,Positive
3,185.wav,Neutral
4,136.wav,Neutral


In [ ]:
sub = sub.sort_values(by='Filename')
sub.head()

,Filename,Class
99,112.wav,Positive
60,113.wav,Negative
84,115.wav,Positive
44,119.wav,Negative
43,123.wav,Neutral


In [ ]:
sub.to_csv('hack_nitp_catboost_v1.csv',index=False)